# Advanced DataBases 24/25


## Goal : Comparing a relational database (MySQL) and a NoSQL database (MongoDB), with specific focus on data modeling, querying, and optimization.
---
## Phase 2
In this phase we will be doing the following:
* Adding more data, (20-25k) to check the results of indexing more truthfully
* Rearanged the structure in MongoDB


#### &copy;Helton Mendonça  | &copy; Riccardo Frattarelli | &copy; Sebastião Cancela   | &copy; Mateusz Kleszcz

### Our Database is from a E-commerce dataset with 3 csv files that are composed like the following:

* List of Orders-This dataset contains purchase information. The information includes ID, Date of Purchase and customer details

* Order Details- This dataset contains order ID, with the order price, quantity,profit, category and subcategory of product

* Sales target-This dataset contains sales target amount and date for each product category

You can find here: https://www.kaggle.com/datasets/benroshan/ecommerce-data?select=Order+Details.csv

In [1]:
import pandas as pd
from pymongo import MongoClient
import mysql.connector
import time
from sqlalchemy import create_engine,text

In [2]:
df_orders_list = pd.read_csv("data_files/List of Orders.csv")
df_order_details = pd.read_csv("data_files/Order Details.csv")
df_sales_target = pd.read_csv("data_files/Sales target.csv")

## Data Cleaning

In [3]:
df_orders_list.dropna(how='all', inplace=True)
df_order_details.dropna(how='all', inplace=True)
df_sales_target.dropna(how='all', inplace=True)

df_orders_list['Order Date'] = pd.to_datetime(df_orders_list['Order Date'], format='%d-%m-%Y')

df_sales_target['Month of Order Date'] = pd.to_datetime(df_sales_target['Month of Order Date'], format='%b-%y')
df_sales_target['Month of Order Date'] = df_sales_target['Month of Order Date'].dt.strftime('%b-%Y')

In [4]:
df_sales_target.dtypes

Month of Order Date     object
Category                object
Target                 float64
dtype: object

In [5]:
# Adding more data
import random
import string

def generate_random_row(df):
    new_order_id = 'B-' + str(random.randint(1, 10**6))  
    new_profit = float(random.randint(-10**3, 10**3))
    new_amount = float(random.randint(1, 10**3))
    new_quantity = int(random.randint(1, 10**2))
    new_category = random.choice(df['Category'].unique())
    new_subcategory = random.choice(df['Sub-Category'].unique())
    return [new_order_id, new_amount, new_profit, new_quantity, new_category, new_subcategory]


# print(new_category)
# print(new_subcategory)
    


In [6]:
existing_order_ids = set(df_order_details['Order ID']) # to avoid duplicates
while len(df_order_details) < 2*10**4:
    if len(df_order_details) % 1000 == 0:  # Print progress every 100 rows
        print(f"Current rows: {len(df_order_details)}")
    new_row = generate_random_row(df_order_details)
    new_order_id = new_row[0]
    if new_order_id not in existing_order_ids:
        
        df_order_details.loc[len(df_order_details)] = new_row
        existing_order_ids.add(new_order_id)
    

Current rows: 2000
Current rows: 3000
Current rows: 4000
Current rows: 5000
Current rows: 6000
Current rows: 7000
Current rows: 8000
Current rows: 9000
Current rows: 10000
Current rows: 11000
Current rows: 12000
Current rows: 13000
Current rows: 14000
Current rows: 15000
Current rows: 16000
Current rows: 17000
Current rows: 18000
Current rows: 19000


In [7]:
print(len(df_order_details))



20000


In [8]:
from datetime import datetime, timedelta

# New list of Indian names (distinct from provided list)
indian_names = ['Arjun', 'Ishita', 'Kabir', 'Meera', 'Neha', 'Rajesh', 'Simran', 'Vikram', 'Sanya', 'Rohan']

# Function to generate random dates within a range
def generate_random_date(start_date, end_date):
    days_range = (end_date - start_date).days
    random_days = random.randint(0, days_range)
    return start_date + pd.Timedelta(days=random_days) # so pra nao ir fora do range da data limite

# Define the date range
start_date = datetime(2014, 4, 1)
end_date = datetime(2018, 3, 31)



# Iterate over Order IDs in df_order_details
existing_order_ids_for_order_list = set(df_orders_list['Order ID'])

for order_id in df_order_details['Order ID']:
    if order_id not in existing_order_ids_for_order_list:
        # Generate new data and append
        random_date = generate_random_date(start_date, end_date)
        random_name = random.choice(indian_names)
        random_state = random.choice(df_orders_list['State'].unique())
        random_city = random.choice(df_orders_list['City'].unique())


        # Update existing_order_ids_for_order_list after adding the order
        existing_order_ids_for_order_list.add(order_id)
        
        df_orders_list.loc[len(df_orders_list)] = [order_id,random_date,random_name, random_state, random_city]






In [9]:
print(len(df_orders_list))
df_orders_list

19000


,Order ID,Order Date,CustomerName,State,City
0,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad
1,B-25602,2018-04-01,Pearl,Maharashtra,Pune
2,B-25603,2018-04-03,Jahan,Madhya Pradesh,Bhopal
3,B-25604,2018-04-03,Divsha,Rajasthan,Jaipur
4,B-25605,2018-04-05,Kasheen,West Bengal,Kolkata
...,...,...,...,...,...
18995,B-312990,2016-06-16,Vikram,Punjab,Kolkata
18996,B-69161,2014-08-22,Simran,Rajasthan,Bhopal
18997,B-371518,2017-08-31,Simran,Himachal Pradesh,Surat
18998,B-597514,2017-09-19,Rajesh,Andhra Pradesh,Chennai


In [10]:
# Step 1: Extract earliest and latest dates
earliest_date = pd.to_datetime(df_orders_list['Order Date']).min()
latest_date = pd.to_datetime(df_orders_list['Order Date']).max()

# Step 2: Generate list of all months in the range
months = pd.date_range(start=earliest_date, end=latest_date, freq='MS')  # 'MS' gives month start dates
month_strings = months.strftime('%b-%Y')  # Format as 'Month-Year'

# Step 3: Generate the new sales target DataFrame
categories = df_order_details['Category'].unique()  # Unique categories from df_order_details

# Set to keep track of added month-category combinations to avoid duplicates
existing_combinations = set()

for month in month_strings:
    # Randomly choose category and ensure no duplicates
    random_category = random.choice(categories)  # Choose a random category
    while (month, random_category) in existing_combinations:
        random_category = random.choice(categories)  # Choose again if duplicate
    
    random_target = float(random.randint(5000, 20000))  # Random target >= 5000
    
    # Append the new entry to the DataFrame
    df_sales_target.loc[len(df_sales_target)] = [month, random_category, random_target]
    
    # Add the combination to the set of existing combinations
    existing_combinations.add((month, random_category))

# Step 4: Sort the DataFrame by Month
df_sales_target = df_sales_target.sort_values(by='Month of Order Date')

# Remove any remaining duplicates just in case
df_sales_target = df_sales_target.drop_duplicates(subset=['Month of Order Date', 'Category'])


In [11]:
df_sales_target

,Month of Order Date,Category,Target
36,Apr-2014,Electronics,15400.0
48,Apr-2015,Furniture,10896.0
60,Apr-2016,Furniture,5640.0
72,Apr-2017,Furniture,5932.0
0,Apr-2018,Furniture,10400.0
...,...,...,...
65,Sep-2016,Electronics,11303.0
77,Sep-2017,Clothing,15625.0
17,Sep-2018,Clothing,14000.0
29,Sep-2018,Electronics,9000.0


In [12]:
print(df_sales_target['Month of Order Date'])

36    Apr-2014
48    Apr-2015
60    Apr-2016
72    Apr-2017
0     Apr-2018
        ...   
65    Sep-2016
77    Sep-2017
17    Sep-2018
29    Sep-2018
5     Sep-2018
Name: Month of Order Date, Length: 84, dtype: object


In [13]:
print(earliest_date)
print(latest_date)

2014-04-01 00:00:00
2019-03-31 00:00:00


In [14]:
print(len(df_sales_target))
print(df_sales_target['Month of Order Date'].unique())
print(df_sales_target['Category'].unique())
print(f"Exact duplicate rows: {df_sales_target.duplicated().sum()}")
print(f"Duplicate rows in ['MonthOfOrder', 'Category']: {df_sales_target.duplicated(subset=['Month of Order Date', 'Category']).sum()}")
print(f"Date range in dataset: {df_sales_target['Month of Order Date'].min()} to {df_sales_target['Month of Order Date'].max()}")


84
['Apr-2014' 'Apr-2015' 'Apr-2016' 'Apr-2017' 'Apr-2018' 'Aug-2014'
 'Aug-2015' 'Aug-2016' 'Aug-2017' 'Aug-2018' 'Dec-2014' 'Dec-2015'
 'Dec-2016' 'Dec-2017' 'Dec-2018' 'Feb-2015' 'Feb-2016' 'Feb-2017'
 'Feb-2018' 'Feb-2019' 'Jan-2015' 'Jan-2016' 'Jan-2017' 'Jan-2018'
 'Jan-2019' 'Jul-2014' 'Jul-2015' 'Jul-2016' 'Jul-2017' 'Jul-2018'
 'Jun-2014' 'Jun-2015' 'Jun-2016' 'Jun-2017' 'Jun-2018' 'Mar-2015'
 'Mar-2016' 'Mar-2017' 'Mar-2018' 'Mar-2019' 'May-2014' 'May-2015'
 'May-2016' 'May-2017' 'May-2018' 'Nov-2014' 'Nov-2015' 'Nov-2016'
 'Nov-2017' 'Nov-2018' 'Oct-2014' 'Oct-2015' 'Oct-2016' 'Oct-2017'
 'Oct-2018' 'Sep-2014' 'Sep-2015' 'Sep-2016' 'Sep-2017' 'Sep-2018']
['Electronics' 'Furniture' 'Clothing']
Exact duplicate rows: 0
Duplicate rows in ['MonthOfOrder', 'Category']: 0
Date range in dataset: Apr-2014 to Sep-2018


In [15]:
# Create a list of all months between April 2014 and April 2019
all_months = pd.date_range(start='2014-04-01', end='2019-04-01', freq='MS').strftime('%b-%Y')

# Identify missing months
missing_months = set(all_months) - set(df_sales_target['Month of Order Date'])
print(f"Missing months: {missing_months}")


Missing months: {'Apr-2019'}


In [16]:
# Manually add the missing month (April 2019)
missing_month_data = {'Month of Order Date': ['Apr-2019'], 'Category': ['Electronics'], 'Target': [10**3]}  # Adjust 'Category' and 'Target' as needed

# Convert to DataFrame
missing_month_df = pd.DataFrame(missing_month_data)

# Append to the existing DataFrame
df_sales_target.loc[len(df_sales_target)] = ['Apr-2019', 'Electronics', 10**3]

# Verify the update
print(f"New dataset length: {len(df_sales_target)}")
print(f"Unique months in new dataset: {df_sales_target['Month of Order Date'].nunique()}")


New dataset length: 84
Unique months in new dataset: 61


In [17]:
df_sales_target['Target'].fillna(method='ffill', inplace=True)
# Verify the update
print(f"New dataset length: {len(df_sales_target)}")
print(f"Unique months in new dataset: {df_sales_target['Month of Order Date'].nunique()}")

New dataset length: 84
Unique months in new dataset: 61


C:\Users\helto\AppData\Local\Temp\ipykernel_21520\190565778.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sales_target['Target'].fillna(method='ffill', inplace=True)
C:\Users\helto\AppData\Local\Temp\ipykernel_21520\190565778.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_sales_target['Target'].fillna(method='ffill', inplace=True)


In [18]:
# Check how many rows exist per unique month and category combination
month_category_counts = df_sales_target.groupby(['Month of Order Date', 'Category']).size().reset_index(name='Row_Count')

# Check the total rows for each month and category combination
print(month_category_counts)

# Check the number of categories per month
category_counts_per_month = df_sales_target.groupby('Month of Order Date')['Category'].nunique().reset_index(name='Num_Categories')
print(category_counts_per_month)


   Month of Order Date     Category  Row_Count
0             Apr-2014  Electronics          1
1             Apr-2015    Furniture          1
2             Apr-2016    Furniture          1
3             Apr-2017    Furniture          1
4             Apr-2018  Electronics          1
..                 ...          ...        ...
79            Sep-2016  Electronics          1
80            Sep-2017     Clothing          1
81            Sep-2018     Clothing          1
82            Sep-2018  Electronics          1
83            Sep-2018    Furniture          1

[84 rows x 3 columns]
   Month of Order Date  Num_Categories
0             Apr-2014               1
1             Apr-2015               1
2             Apr-2016               1
3             Apr-2017               1
4             Apr-2018               2
..                 ...             ...
56            Sep-2014               1
57            Sep-2015               1
58            Sep-2016               1
59            Sep-2017 

## Queries

In [19]:
queries = [
    {"description": "Find all positive Profit in orders with less than 500 amount"},
    
    {"description": "Find how many orders came from Mumbai"},
    
    {"description": "Identify Most Profitable States"},
    
    {"description": "Monitor Sales Target Achievement Over Time"}
]

## MongoDB

##### Create Database

In [20]:
client = MongoClient("mongodb://localhost:27017")

client.drop_database("Project_DB")
print("Database dropped")

db = client['Project_DB']

Database dropped


##### Create collections

In [21]:
db["orders_list"].drop()
db["order_details"].drop()
db["sales_target"].drop()
db['orders'].drop()
print("Collections dropped")

# collection_orders_list = db["orders_list"]
# collection_order_details = db["order_details"]
collection_orders = db['orders']
collection_sales_target = db["sales_target"]

Collections dropped


In [22]:
### --- Old version of the collections ---

# db.create_collection("orders_list", validator={
#     '$jsonSchema':{
#         'bsonType': 'object',
#         'required': ["Order ID", "Order Date", "CustomerName", "State", "City"],
#         'properties':{
#             "Order ID": {
#           'bsonType': "string",
#           'description': "Must be a string and is the unique identifier for orders"
#             },
#             'Order Date':{
#                 'bsonType':'date',
#                 'description': 'Must be a date'
#             },
#             'CustomerName':{
#                 'bsonType':'string',
#                 'description': 'Must be a string'
#             },
#             'State':{
#                 'bsonType':'string',
#                 'description': 'Must be a string'
#             },
#             'City':{
#                 'bsonType':'string',
#                 'description': 'Must be a string'
#             }
#         }
        
#     }
# })

# db.create_collection("order_details", validator={
#     '$jsonSchema':{
#         "bsonType": "object",
#         "required": ["Order ID", "Details"],
#         "properties": {
#             "Order ID": {
#                 "bsonType": "string",
#                 "description": "Must reference an existing Order ID in ordersList"
#             },
#             "Details": {
#                 "bsonType": "array",
#                 "items": {
#                     "bsonType": "object",
#                     "required": ["Amount", "Profit", "Quantity", "Category", "SubCategory"],
#                     "properties": {
#                         "Amount": {
#                             "bsonType": "double",
#                             "description": "Must be a non-negative double"
#                         },
#                         "Profit": {
#                             "bsonType": "double",
#                             "description": "Must be a double"
#                         },
#                         "Quantity": {
#                             "bsonType": "int",
#                             "description": "Must be a non-negative integer"
#                         },
#                         "Category": {
#                             "bsonType": "string",
#                             "description": "Must be a string"
#                         },
#                         "SubCategory": {
#                             "bsonType": "string",
#                             "description": "Must be a string"
#                         },
#                         },
#                     }
#                 }
#             }
#         }
#     }
# )

# db.create_collection("sales_target", validator={
#     '$jsonSchema':{
#         "bsonType": "object",
#         "required": ["Month of Order Date", "SalesTargets"],
#         "properties": {
#             "Month of Order Date": {
#                 "bsonType": "string",
#                 "description": "Must be a string of format %b-%y"
#                 },
#             "SalesTargets": {
#                 "bsonType": "array",
#                 "items": {
#                     "bsonType": "object",
#                     "required": ["Category", "Target"],
#                     "properties": {
#                         "Category": {
#                             "bsonType": "string",
#                             "description": "Must be a string"
#                         },
#                         "Target": {
#                             "bsonType": "double",
#                             "description": "Must be a non-negative double"
#                         }
#                     }
#                 }
#             }
#         }
#     }
# })

In [23]:
### ---- New version of the collections------

db.create_collection("orders", validator={
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ["Order ID", "Order Date", "CustomerName", "State", "City", "Details"],
        'properties': {
            "Order ID": {
                'bsonType': "string",
                'description': "Must be a string and is the unique identifier for orders"
            },
            'Order Date': {
                'bsonType': 'date',
                'description': 'Must be a date'
            },
            'CustomerName': {
                'bsonType': 'string',
                'description': 'Must be a string'
            },
            'State': {
                'bsonType': 'string',
                'description': 'Must be a string'
            },
            'City': {
                'bsonType': 'string',
                'description': 'Must be a string'
            },
            "Details": {
                'bsonType': "array",
                'items': {
                    'bsonType': "object",
                    'required': ["Amount", "Profit", "Quantity", "Category", "SubCategory"],
                    'properties': {
                        "Amount": {
                            'bsonType': "double",
                            'description': "Must be a non-negative double"
                        },
                        "Profit": {
                            'bsonType': "double",
                            'description': "Must be a double"
                        },
                        "Quantity": {
                            'bsonType': "int",
                            'description': "Must be a non-negative integer"
                        },
                        "Category": {
                            'bsonType': "string",
                            'description': "Must be a string"
                        },
                        "SubCategory": {
                            'bsonType': "string",
                            'description': "Must be a string"
                        }
                    }
                },
                'description': "Array of order details"
            }
        }
    }
})


db.create_collection("sales_target", validator={
    '$jsonSchema':{
        "bsonType": "object",
        "required": ["Month of Order Date", "SalesTargets"],
        "properties": {
            "Month of Order Date": {
                "bsonType": "string",
                "description": "Must be a string of format %b-%y"
                },
            "SalesTargets": {
                "bsonType": "array",
                "items": {
                    "bsonType": "object",
                    "required": ["Category", "Target"],
                    "properties": {
                        "Category": {
                            "bsonType": "string",
                            "description": "Must be a string"
                        },
                        "Target": {
                            "bsonType": "double",
                            "description": "Must be a non-negative double"
                        }
                    }
                }
            }
        }
    }
})

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Project_DB'), 'sales_target')

##### Insert Data

In [24]:
### --- Old version of the collections ---

# initial_count = collection_orders_list.count_documents({})
# print(f"# documents: {initial_count}") 
# try:
#     data_dict = df_orders_list.to_dict(orient="records")
#     collection_orders_list.insert_many(data_dict, ordered=False)
    
#     order_details_data = []

#     grouped_order_details = df_order_details.groupby('Order ID')
    
#     for order_id, group in grouped_order_details:

#         order_details = {
#             "Order ID": order_id,
#             "Details": []
#         }
        
#         for _, row in group.iterrows():
#             detail = {
#                 "Amount": row['Amount'],
#                 "Profit": row['Profit'],
#                 "Quantity": row['Quantity'],
#                 "Category": row['Category'],
#                 "SubCategory": row['Sub-Category']
#             }
#             order_details["Details"].append(detail)
        
#         order_details_data.append(order_details)

#     db.order_details.insert_many(order_details_data)

#     sales_target_data = []

#     grouped_sales_target = df_sales_target.groupby('Month of Order Date')
    
    
#     for month, group in grouped_sales_target:

#         sales_target = {
#             "Month of Order Date": month,
#             "SalesTargets": []
#         }
#         for _, row in group.iterrows():
#             targets = {
#                 "Category": row['Category'],
#                 "Target": row['Target'],
#             }
#             sales_target["SalesTargets"].append(targets)
        
#         sales_target_data.append(sales_target)
        
#     db.sales_target.insert_many(sales_target_data)
    
# except Exception as e:
#     print("document failed to insert")


# print(f"#orders documents: {collection_orders_list.count_documents({})}") 
# print(f"#orders details documents: {collection_order_details.count_documents({})}") 
# print(f"#sales target documents: {collection_sales_target.count_documents({})}") 

In [25]:
### ---- New version of the collections------

# Verificando o número inicial de documentos
initial_count = collection_orders.count_documents({})
print(f"# documents: {initial_count}")

try:
    # Criando uma lista para armazenar os documentos formatados
    orders_data = []

    # Agrupando os detalhes dos pedidos pelo "Order ID"
    grouped_order_details = df_order_details.groupby('Order ID')

    # Convertendo os dados principais de `df_orders_list` para um dicionário para acesso eficiente
    orders_list_dict = df_orders_list.set_index("Order ID").to_dict(orient="index")

    # Iterando sobre cada grupo de detalhes
    for order_id, group in grouped_order_details:
        # Criando o documento principal com base nos dados de orders_list
        if order_id in orders_list_dict:
            order = orders_list_dict[order_id]
            order_document = {
                "Order ID": order_id,
                "Order Date": order["Order Date"],
                "CustomerName": order["CustomerName"],
                "State": order["State"],
                "City": order["City"],
                "Details": []  # Adicionando a chave para detalhes
            }

            # Adicionando os detalhes do pedido
            for _, row in group.iterrows():
                detail = {
                    "Amount": row['Amount'],
                    "Profit": row['Profit'],
                    "Quantity": row['Quantity'],
                    "Category": row['Category'],
                    "SubCategory": row['Sub-Category']
                }
                order_document["Details"].append(detail)

            # Adicionando o documento completo à lista de inserção
            orders_data.append(order_document)

    # Inserindo os documentos na coleção "orders"
    collection_orders.insert_many(orders_data, ordered=False)

    # Inserindo os dados de sales_target sem alterações
    sales_target_data = []

    grouped_sales_target = df_sales_target.groupby('Month of Order Date')

    for month, group in grouped_sales_target:
        sales_target = {
            "Month of Order Date": month,
            "SalesTargets": []
        }
        for _, row in group.iterrows():
            targets = {
                "Category": row['Category'],
                "Target": row['Target'],
            }
            sales_target["SalesTargets"].append(targets)

        sales_target_data.append(sales_target)

    db.sales_target.insert_many(sales_target_data)

except Exception as e:
    print("Document failed to insert:", e)

# Exibindo o número final de documentos
print(f"#orders documents: {collection_orders.count_documents({})}")
print(f"#sales target documents: {collection_sales_target.count_documents({})}") 


# documents: 0
#orders documents: 19000
#sales target documents: 61


##### Execute queries

In [26]:
# ### --- Old version of the collections ---

# head_count = 5
# mongo_times = []
# for q in queries:
#     print(f"Executing: {q['description']}")
#     start_time = time.time()
    
#     if q['description'] == "Find all positive Profit in orders with less than 500 amount":
#         query = [
#             {
#                 "$unwind": "$Details"  
#             },
#             {
#                 "$match": {
#                     "Details.Profit": { "$gt": 0 }, 
#                     "Details.Amount": {"$lt": 500}
#                 }
#             },
#             {
#                 "$sort": {
#                     "Details.Profit": -1  
#                 }
#             },
#             {
#                 "$project": {
#                     "Order ID": 1,  
#                     "Item details": "$Details",   
#                     "_id": 0        
#                 }
#             }
#         ]
        
#         result = collection_order_details.aggregate(query)
        
#     elif q['description'] == "Find how many orders came from Mumbai":
#         query = [
#             {
#                 "$match": {"City": "Mumbai"} 
#             },
#             {
#                 "$count": "total_orders" 
#             }
#         ]
           
        
#         result = collection_orders_list.aggregate(query)
        
#     elif q['description'] == "Identify Most Profitable States":
#         query = [
#             {
#                 "$lookup": {
#                     "from": "orders_list",  
#                     "localField": "Order ID",  
#                     "foreignField": "Order ID",  
#                     "as": "order_info"  
#                 }
#             },
#             {
#                 "$unwind": "$order_info"  
#             },
#             {
#                 "$unwind": "$Details"  
#             },
#             {
#                 "$group": {
#                     "_id": "$order_info.State",  
#                     "TotalProfit": { "$sum": "$Details.Profit"},
#                     "TotalOrders": {"$sum": 1}
#                 }
#             },
#             {
#                 "$match": {
#                     "TotalProfit": { "$gt": 0 } 
#                 }
#             },
#             {
#                 "$sort": {
#                     "TotalProfit": -1  
#                 }
#             },
#             {
#                 "$project": {
#                     "_id": 0,  
#                     "State": "$_id",  
#                     "TotalProfit": 1,  
#                     "TotalOrders": 1
#                 }
#             }
#         ]
        
#         result = collection_order_details.aggregate(query)
        
#         db["states_profit"].drop()

#         #collection_orders_list = db["states_profit"]
        
#         db.create_collection("states_profit", validator={
#             '$jsonSchema':{
#                 'bsonType': 'object',
#                 'required': ['State', 'TotalProfit', 'TotalOrders'],
#                 'properties':{
#                     'State':{
#                         'bsonType':'string',
#                         'description': 'Must be a string'
#                     },
#                     'TotalProfit':{
#                         'bsonType':'double',
#                         'description': 'Must be a non-negative double'
#                     },
#                     'TotalOrders':{
#                         'bsonType':'int',
#                         'description': 'Must be a non-negative integer'
#                     }
#                 }
#             }
#         })
        
#         db['states_profit'].insert_many(list(result))
        
#         result = collection_order_details.aggregate(query) # For some reason it's necessary to repeat this line
        
#     elif q["description"] == "Monitor Sales Target Achievement Over Time":
#         first_query = [
#             {
#                 "$lookup": {
#                     "from": "orders_list",
#                     "localField": "Order ID",
#                     "foreignField": "Order ID",
#                     "as": "order_info"
#                 }
#             },
#             {
#                 "$unwind": "$order_info"  
#             },
#             {
#                 "$unwind": "$Details" 
#             },
#             {
#                 "$group": {
#                     "_id": {
#                         "Month": {"$dateToString": {"format": "%b-%Y", "date": "$order_info.Order Date"}},  
#                         "Category": "$Details.Category"
#                     },
#                     "TotalAmountSpent": {"$sum": "$Details.Amount"} 
#                 }
#             },
#             {
#                 "$lookup": {
#                     "from": "sales_target",  
#                     "localField": "_id.Month",
#                     "foreignField": "Month of Order Date",
#                     "as": "targets"
#                 }
#             },
#             {
#                 "$unwind": "$targets" 
#             },
#             {
#                 "$unwind": "$targets.SalesTargets"
#             },
#             {
#                 "$match": {
#                     "$expr": {
#                         "$and": [
#                             {"$eq": ["$_id.Category", "$targets.SalesTargets.Category"]},  
#                             {"$eq": ["$_id.Month", "$targets.Month of Order Date"]}  
#                         ]
#                     }
#                 }
#             },
#             {
#                 "$project": {
#                     "_id": 0,  
#                     "Month": "$_id.Month", 
#                     "Category": "$_id.Category", 
#                     "TargetAchieved": {
#                         "$cond": {
#                             "if": {"$gt": ["$TotalAmountSpent", "$targets.SalesTargets.Target"]}, 
#                             "then": True, 
#                             "else": False 
#                         }
#                     }
#                 }
#             }
            
#         ]

#         result = collection_order_details.aggregate(first_query)
        
        
#         db["target_achievement"].drop()
        
#         db.create_collection("target_achievement", validator={
#             '$jsonSchema':{
#                 'bsonType': 'object',
#                 'required': ['Month', 'Category', 'TargetAchieved'],
#                 'properties':{
#                     'Month':{
#                         'bsonType':'string',
#                         'description': 'Must be a string'
#                     },
#                     'Category':{
#                         'bsonType':'string',
#                         'description': 'Must be a non-negative string'
#                     },
#                     'TargetAchieved':{
#                         'bsonType':'bool',
#                         'description': 'Must be a bool'
#                     }
#                 }
#             }
#         })
        
#         db['target_achievement'].insert_many(list(result))
        
#         second_query = [
#             {
#                 "$match": {
#                     "TargetAchieved": True 
#                 }
#             },
#             {
#                 "$count": "Targets Achieved" 
#             }
#         ]
        
#         result = db['target_achievement'].aggregate(second_query)
        
#     else:
#         print(f"Unhandled query: {q['description']}")
#         continue

#     end_time = time.time()
    
#     list_result = list(result)
#     mongo_times.append(end_time - start_time)
    
#     print(f"Results for '{q['description']}' (showing up to {head_count} results):")
#     for i, doc in enumerate(list_result[:head_count]):
#         print(doc)
    
#     if len(list_result) > head_count:
#         print(f"...and {len(list_result) - head_count} more results")
    
#     print(f"Query executed in {end_time - start_time:.4f} seconds\n")

In [27]:
# Controladores
head_count = 5
mongo_times = []

# Consultas
for q in queries:
    print(f"Executing: {q['description']}")
    start_time = time.time()

    if q['description'] == "Find all positive Profit in orders with less than 500 amount":
        query = [
            {"$unwind": "$Details"},
            {"$match": {"Details.Profit": {"$gt": 0}, "Details.Amount": {"$lt": 500}}},
            {"$sort": {"Details.Profit": -1}},
            {"$project": {"Order ID": 1, "Item details": "$Details", "_id": 0}}
        ]
        result = db["orders"].aggregate(query)

    elif q['description'] == "Find how many orders came from Mumbai":
        query = [
            {"$match": {"City": "Mumbai"}},
            {"$count": "total_orders"}
        ]
        result = db["orders"].aggregate(query)

    elif q['description'] == "Identify Most Profitable States":
        query = [
            {"$unwind": "$Details"},
            {"$group": {
                "_id": "$State",
                "TotalProfit": {"$sum": "$Details.Profit"},
                "TotalOrders": {"$sum": 1}
            }},
            {"$match": {"TotalProfit": {"$gt": 0}}},
            {"$sort": {"TotalProfit": -1}},
            {"$project": {"_id": 0, "State": "$_id", "TotalProfit": 1, "TotalOrders": 1}}  # Exclude _id field
        ]
        result = db['orders'].aggregate(query)

        # Create the states_profit collection with a validator (optional)
        db["states_profit"].drop()  # Drop the collection if it exists
        db.create_collection("states_profit", validator={
            '$jsonSchema':{
                'bsonType': 'object',
                'required': ['State', 'TotalProfit', 'TotalOrders'],
                'properties':{
                    'State':{
                        'bsonType':'string',
                        'description': 'Must be a string'
                    },
                    'TotalProfit':{
                        'bsonType':'double',
                        'description': 'Must be a non-negative double'
                    },
                    'TotalOrders':{
                        'bsonType':'int',
                        'description': 'Must be a non-negative integer'
                    }
                }
            }
        })
        
        # Insert the aggregated data into the 'states_profit' collection
        db['states_profit'].insert_many(list(result))
        
        # Retrieve the data to display (without '_id' field)
        list_result = db['states_profit'].find({}, {"_id": 0})  # Exclude _id field from the query
    
        # Print results with head_count
        print(f"Results for '{q['description']}' (up to {head_count}):")
        for i, doc in enumerate(list_result[:head_count]):
            print(doc)
        
        # Print total count
        # print(f"Total number of states: {len(list_result)}")


            # db["states_profit"].drop()
            # db["states_profit"].insert_many(list(result))  # Inserção para visualização futura

    elif q["description"] == "Monitor Sales Target Achievement Over Time":
        first_query = [
            {"$unwind": "$Details"},
            {"$group": {
                "_id": {
                    "Month": {"$dateToString": {"format": "%b-%Y", "date": "$Order Date"}},
                    "Category": "$Details.Category"
                },
                "TotalAmountSpent": {"$sum": "$Details.Amount"}
            }},
            {"$lookup": {
                "from": "sales_target",
                "localField": "_id.Month",
                "foreignField": "Month of Order Date",
                "as": "targets"
            }},
            {"$unwind": "$targets"},
            {"$unwind": "$targets.SalesTargets"},
            {"$match": {
                "$expr": {
                    "$and": [
                        {"$eq": ["$_id.Category", "$targets.SalesTargets.Category"]},
                        {"$eq": ["$_id.Month", "$targets.Month of Order Date"]}
                    ]
                }
            }},
            {"$project": {
                "_id": 0,
                "Month": "$_id.Month",
                "Category": "$_id.Category",
                "TargetAchieved": {
                    "$cond": {
                        "if": {"$gt": ["$TotalAmountSpent", "$targets.SalesTargets.Target"]},
                        "then": True, "else": False
                    }
                }
            }}
        ]
        result = db["orders"].aggregate(first_query)

        db["target_achievement"].drop()
        db["target_achievement"].insert_many(list(result))  # Inserção para visão futura

        second_query = [
            {"$match": {"TargetAchieved": True}},
            {"$count": "TargetsAchieved"}
        ]
        result_achieved = db["target_achievement"].aggregate(second_query)
        
        count_result = list(db['target_achievement'].aggregate(second_query))
        print('count result: ', count_result)
        if count_result:
            achieved_count = count_result[0]['TargetsAchieved']
            print(f"Number of months where the category target was achieved: {achieved_count}\n  Remember same dates might have different targets for different categories")
            
        else:
            print("No targets achieved.")

    else:
        print(f"Unhandled query: {q['description']}")
        continue

    end_time = time.time()
    mongo_times.append(end_time - start_time)

    # Impressão de resultados
    list_result = list(result)
    print(f"Results for '{q['description']}' (up to {head_count}):")
    for i, doc in enumerate(list_result[:head_count]):
        print(doc)

    if len(list_result) > head_count:
        print(f"...and {len(list_result) - head_count} more results")

    print(f"Query executed in {end_time - start_time:.4f} seconds\n")

Executing: Find all positive Profit in orders with less than 500 amount
Results for 'Find all positive Profit in orders with less than 500 amount' (up to 5):
{'Order ID': 'B-150914', 'Item details': {'Amount': 230.0, 'Profit': 1000.0, 'Quantity': 38, 'Category': 'Furniture', 'SubCategory': 'Chairs'}}
{'Order ID': 'B-950416', 'Item details': {'Amount': 236.0, 'Profit': 1000.0, 'Quantity': 74, 'Category': 'Electronics', 'SubCategory': 'Furnishings'}}
{'Order ID': 'B-455900', 'Item details': {'Amount': 466.0, 'Profit': 999.0, 'Quantity': 90, 'Category': 'Furniture', 'SubCategory': 'Electronic Games'}}
{'Order ID': 'B-669254', 'Item details': {'Amount': 67.0, 'Profit': 999.0, 'Quantity': 65, 'Category': 'Furniture', 'SubCategory': 'Skirt'}}
{'Order ID': 'B-903647', 'Item details': {'Amount': 297.0, 'Profit': 999.0, 'Quantity': 68, 'Category': 'Furniture', 'SubCategory': 'Accessories'}}
...and 5444 more results
Query executed in 0.0946 seconds

Executing: Find how many orders came from Mumb

In [28]:
collection = db['states_profit']

documents = collection.find()

for document in documents:
    print(document)


{'_id': ObjectId('675b1eda095caebfa6a865c6'), 'TotalProfit': 40908.0, 'TotalOrders': 1083, 'State': 'Bihar'}
{'_id': ObjectId('675b1eda095caebfa6a865c7'), 'TotalProfit': 31567.0, 'TotalOrders': 1049, 'State': 'Andhra Pradesh'}
{'_id': ObjectId('675b1eda095caebfa6a865c8'), 'TotalProfit': 28017.0, 'TotalOrders': 962, 'State': 'Goa'}
{'_id': ObjectId('675b1eda095caebfa6a865c9'), 'TotalProfit': 24938.0, 'TotalOrders': 1028, 'State': 'Rajasthan'}
{'_id': ObjectId('675b1eda095caebfa6a865ca'), 'TotalProfit': 12094.0, 'TotalOrders': 1087, 'State': 'Gujarat'}
{'_id': ObjectId('675b1eda095caebfa6a865cb'), 'TotalProfit': 11240.0, 'TotalOrders': 1029, 'State': 'Haryana'}
{'_id': ObjectId('675b1eda095caebfa6a865cc'), 'TotalProfit': 3464.0, 'TotalOrders': 1286, 'State': 'Maharashtra'}
{'_id': ObjectId('675b1eda095caebfa6a865cd'), 'TotalProfit': 1724.0, 'TotalOrders': 1030, 'State': 'Delhi'}
{'_id': ObjectId('675b1eda095caebfa6a865ce'), 'TotalProfit': 390.0, 'TotalOrders': 1016, 'State': 'West Bengal

In [29]:
collection = db['target_achievement']

documents = collection.find()

for document in documents:
    print(document)


{'_id': ObjectId('675b1edb095caebfa6a865cf'), 'Month': 'Jan-2015', 'Category': 'Furniture', 'TargetAchieved': True}
{'_id': ObjectId('675b1edb095caebfa6a865d0'), 'Month': 'Jul-2014', 'Category': 'Electronics', 'TargetAchieved': True}
{'_id': ObjectId('675b1edb095caebfa6a865d1'), 'Month': 'Oct-2017', 'Category': 'Clothing', 'TargetAchieved': True}
{'_id': ObjectId('675b1edb095caebfa6a865d2'), 'Month': 'Apr-2016', 'Category': 'Furniture', 'TargetAchieved': True}
{'_id': ObjectId('675b1edb095caebfa6a865d3'), 'Month': 'Feb-2016', 'Category': 'Clothing', 'TargetAchieved': True}
{'_id': ObjectId('675b1edb095caebfa6a865d4'), 'Month': 'Oct-2015', 'Category': 'Clothing', 'TargetAchieved': True}
{'_id': ObjectId('675b1edb095caebfa6a865d5'), 'Month': 'May-2018', 'Category': 'Furniture', 'TargetAchieved': False}
{'_id': ObjectId('675b1edb095caebfa6a865d6'), 'Month': 'Nov-2018', 'Category': 'Clothing', 'TargetAchieved': True}
{'_id': ObjectId('675b1edb095caebfa6a865d7'), 'Month': 'Nov-2018', 'Categ

### CREATING INDEXING ON MongoDB

In [30]:
# Indexes for the "orders" collection
db["orders"].create_index([("Details.Profit", 1), ("Details.Amount", 1)], name="idx_profit_amount")
db["orders"].create_index([("City", 1)], name="idx_city")
db["orders"].create_index([("State", 1)], name="idx_orders_list_state")
db["orders"].create_index([("Details.Profit", 1)], name="idx_order_details_profit")
db["orders"].create_index([("Order Date", 1)], name="idx_order_date_month")

# Indexes for the "sales_target" collection
db["sales_target"].create_index([("SalesTargets.Category", 1), ("Month of Order Date", 1)], name="idx_sales_target_category_month")


'idx_sales_target_category_month'

In [31]:
# Controladores
head_count = 5
mongo_times_ind = []
query_times = []

# Consultas
for q in queries:
    print(f"Executing: {q['description']}")
    start_time = time.time()

    if q['description'] == "Find all positive Profit in orders with less than 500 amount":
        query = [
            {"$unwind": "$Details"},
            {"$match": {"Details.Profit": {"$gt": 0}, "Details.Amount": {"$lt": 500}}},
            {"$sort": {"Details.Profit": -1}},
            {"$project": {"Order ID": 1, "Item details": "$Details", "_id": 0}}
        ]
        result = db["orders"].aggregate(query)

    elif q['description'] == "Find how many orders came from Mumbai":
        query = [
            {"$match": {"City": "Mumbai"}},
            {"$count": "total_orders"}
        ]
        result = db["orders"].aggregate(query)

    elif q['description'] == "Identify Most Profitable States":
        query = [
            {"$unwind": "$Details"},
            {"$group": {
                "_id": "$State",
                "TotalProfit": {"$sum": "$Details.Profit"},
                "TotalOrders": {"$sum": 1}
            }},
            {"$match": {"TotalProfit": {"$gt": 0}}},
            {"$sort": {"TotalProfit": -1}},
            {"$project": {"_id": 0, "State": "$_id", "TotalProfit": 1, "TotalOrders": 1}}  # Exclude _id field
        ]
        result = db['orders'].aggregate(query)

        # Create the states_profit collection with a validator (optional)
        db["states_profit"].drop()  # Drop the collection if it exists
        db.create_collection("states_profit", validator={
            '$jsonSchema':{
                'bsonType': 'object',
                'required': ['State', 'TotalProfit', 'TotalOrders'],
                'properties':{
                    'State':{
                        'bsonType':'string',
                        'description': 'Must be a string'
                    },
                    'TotalProfit':{
                        'bsonType':'double',
                        'description': 'Must be a non-negative double'
                    },
                    'TotalOrders':{
                        'bsonType':'int',
                        'description': 'Must be a non-negative integer'
                    }
                }
            }
        })
        
        # Insert the aggregated data into the 'states_profit' collection
        db['states_profit'].insert_many(list(result))
        
        # Retrieve the data to display (without '_id' field)
        list_result = db['states_profit'].find({}, {"_id": 0})  # Exclude _id field from the query
    
        # Print results with head_count
        print(f"Results for '{q['description']}' (up to {head_count}):")
        for i, doc in enumerate(list_result[:head_count]):
            print(doc)
        
        # Print total count
        # print(f"Total number of states: {len(list_result)}")


            # db["states_profit"].drop()
            # db["states_profit"].insert_many(list(result))  # Inserção para visualização futura

    elif q["description"] == "Monitor Sales Target Achievement Over Time":
        first_query = [
            {"$unwind": "$Details"},
            {"$group": {
                "_id": {
                    "Month": {"$dateToString": {"format": "%b-%Y", "date": "$Order Date"}},
                    "Category": "$Details.Category"
                },
                "TotalAmountSpent": {"$sum": "$Details.Amount"}
            }},
            {"$lookup": {
                "from": "sales_target",
                "localField": "_id.Month",
                "foreignField": "Month of Order Date",
                "as": "targets"
            }},
            {"$unwind": "$targets"},
            {"$unwind": "$targets.SalesTargets"},
            {"$match": {
                "$expr": {
                    "$and": [
                        {"$eq": ["$_id.Category", "$targets.SalesTargets.Category"]},
                        {"$eq": ["$_id.Month", "$targets.Month of Order Date"]}
                    ]
                }
            }},
            {"$project": {
                "_id": 0,
                "Month": "$_id.Month",
                "Category": "$_id.Category",
                "TargetAchieved": {
                    "$cond": {
                        "if": {"$gt": ["$TotalAmountSpent", "$targets.SalesTargets.Target"]},
                        "then": True, "else": False
                    }
                }
            }}
        ]
        result = db["orders"].aggregate(first_query)

        db["target_achievement"].drop()
        db["target_achievement"].insert_many(list(result))  # Inserção para visão futura

        second_query = [
            {"$match": {"TargetAchieved": True}},
            {"$count": "TargetsAchieved"}
        ]
        result_achieved = db["target_achievement"].aggregate(second_query)
        
        count_result = list(db['target_achievement'].aggregate(second_query))
        print('count result: ', count_result)
        if count_result:
            achieved_count = count_result[0]['TargetsAchieved']
            print(f"Number of months where the category target was achieved: {achieved_count}\n  Remember same dates might have different targets for different categories")
            
        else:
            print("No targets achieved.")

    else:
        print(f"Unhandled query: {q['description']}")
        continue

    end_time = time.time()
    mongo_times_ind.append(end_time - start_time)

    # Impressão de resultados
    list_result = list(result)
    print(f"Results for '{q['description']}' (up to {head_count}):")
    for i, doc in enumerate(list_result[:head_count]):
        print(doc)

    if len(list_result) > head_count:
        print(f"...and {len(list_result) - head_count} more results")

    print(f"Query executed in {end_time - start_time:.4f} seconds\n")
    
    
    query_times.append(mongo_times_ind)

average_times = [sum(times) / len(times) for times in zip(*query_times)]

for i, q in enumerate(queries):
    print(f"Average execution time for '{q['description']}': {average_times[i]:.4f} seconds")

Executing: Find all positive Profit in orders with less than 500 amount
Results for 'Find all positive Profit in orders with less than 500 amount' (up to 5):
{'Order ID': 'B-150914', 'Item details': {'Amount': 230.0, 'Profit': 1000.0, 'Quantity': 38, 'Category': 'Furniture', 'SubCategory': 'Chairs'}}
{'Order ID': 'B-950416', 'Item details': {'Amount': 236.0, 'Profit': 1000.0, 'Quantity': 74, 'Category': 'Electronics', 'SubCategory': 'Furnishings'}}
{'Order ID': 'B-455900', 'Item details': {'Amount': 466.0, 'Profit': 999.0, 'Quantity': 90, 'Category': 'Furniture', 'SubCategory': 'Electronic Games'}}
{'Order ID': 'B-669254', 'Item details': {'Amount': 67.0, 'Profit': 999.0, 'Quantity': 65, 'Category': 'Furniture', 'SubCategory': 'Skirt'}}
{'Order ID': 'B-903647', 'Item details': {'Amount': 297.0, 'Profit': 999.0, 'Quantity': 68, 'Category': 'Furniture', 'SubCategory': 'Accessories'}}
...and 5444 more results
Query executed in 0.0964 seconds

Executing: Find how many orders came from Mumb

## MySQL

##### Create Database

In [32]:

username = 'root'
password = '1234'
host = 'localhost'
port = '3306'
database_name = 'P1'


db_url = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}?use_pure=True'

engine = create_engine(db_url)


with engine.connect() as connection:
    connection.execute(text(f"DROP DATABASE IF EXISTS {database_name}"))
    connection.execute(text(f"CREATE DATABASE {database_name}"))


db_url = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(db_url)


##### Create tables

In [33]:
with engine.connect() as connection:
    
    # RUN THE FOLLOWING ONLY IF IT IS NOT THE FIRST TIME
    #connection.execute(text("ALTER TABLE order_details DROP FOREIGN KEY order_details_ibfk_1"))
    connection.execute(text("DROP TABLE IF EXISTS orders_list"))
    connection.execute(text("DROP TABLE IF EXISTS order_details"))
    connection.execute(text("DROP TABLE IF EXISTS sales_target"))
    connection.execute(text("DROP TABLE IF EXISTS categories"))
    connection.execute(text("DROP TABLE IF EXISTS sub_categories"))
    print("Collections dropped")

    connection.execute(text("""CREATE TABLE orders_list (
                                `Order ID` VARCHAR(255) PRIMARY KEY,
                                `Order DATE` DATE NOT NULL,          
                                CustomerNAME VARCHAR(255) NOT NULL, 
                                State VARCHAR(255) NOT NULL, 
                                City VARCHAR(255) NOT NULL
                            )
                        """))
    
    connection.execute(text("""CREATE TABLE categories (
                                CategoryID INT PRIMARY KEY,           
                                ProductCategory VARCHAR(100) NOT NULL
                            )
                            """))
    
    connection.execute(text("""CREATE TABLE sub_categories (
                                SubProductCategoryID INT AUTO_INCREMENT PRIMARY KEY,           
                                ProductCategory VARCHAR(100) NOT NULL,       
                                ProductSubcategory VARCHAR(100) NOT NULL,
                                CategoryID INT NOT NULL,
                                FOREIGN KEY (CategoryID) 
                                    REFERENCES categories(CategoryID)   
                                    ON DELETE CASCADE 
                                    ON UPDATE CASCADE
                            )
                            """))

    connection.execute(text("""CREATE TABLE order_details (
                                DetailID INT AUTO_INCREMENT PRIMARY KEY,     
                                `Order ID` VARCHAR(255) NOT NULL,    
                                SubProductCategoryID INT NOT NULL,       
                                CategoryID INT NOT NULL,             
                                Amount INT NOT NULL,
                                Quantity INT NOT NULL,                 
                                Profit FLOAT(12,2) NOT NULL,                 
                                FOREIGN KEY (`Order ID`) 
                                    REFERENCES orders_list(`Order ID`)       
                                    ON DELETE CASCADE 
                                    ON UPDATE CASCADE,
                                FOREIGN KEY (CategoryID) 
                                    REFERENCES categories(CategoryID)   
                                    ON DELETE CASCADE 
                                    ON UPDATE CASCADE,
                                FOREIGN KEY (SubProductCategoryID) 
                                    REFERENCES sub_categories(SubProductCategoryID)   
                                    ON DELETE CASCADE 
                                    ON UPDATE CASCADE
                            )
                            """))

    connection.execute(text("""CREATE TABLE sales_target (
                                CategoryID INT NOT NULL,            
                                MonthOfOrder DATE NOT NULL,                
                                Target DECIMAL(10,2) NOT NULL,  
                                PRIMARY KEY (CategoryID, MonthOfOrder), 
                                FOREIGN KEY (CategoryID) 
                                    REFERENCES categories(CategoryID)
                                    ON DELETE CASCADE
                                    ON UPDATE CASCADE
                            )
                            """))


    

Collections dropped


##### Insert Data

In [34]:
df_categories = df_order_details[['Category', 'Sub-Category']]
df_categories.columns = ['ProductCategory', 'ProductSubcategory']
df_categories = df_categories.drop_duplicates(subset=['ProductCategory', 'ProductSubcategory'], keep='first')
category_mapping = {category: idx + 1 for idx, category in enumerate(df_categories['ProductCategory'].unique())}
df_categories['CategoryID'] = df_categories['ProductCategory'].map(category_mapping)
df_categories_cat = df_categories[['CategoryID', 'ProductCategory']]
df_categories_cat = df_categories_cat.drop_duplicates(subset=['ProductCategory'])
df_categories_sub = df_categories[['ProductCategory', 'ProductSubcategory', 'CategoryID']]
df_orders_list.to_sql('orders_list', con=engine, if_exists='append', index=False, method='multi')
df_categories_cat.to_sql('categories', con=engine, if_exists='append', index=False, method='multi')
df_categories_sub.to_sql('sub_categories', con=engine, if_exists='append', index=False, method='multi')

51

In [35]:
query_categories = """
    SELECT CategoryID, SubProductCategoryID, ProductCategory, ProductSubcategory
    FROM sub_categories
"""
categories_df = pd.read_sql(query_categories, con=engine)
df_order_details = df_order_details.merge(
    categories_df,
    how='left',
    left_on=['Category', 'Sub-Category'], 
    right_on=['ProductCategory', 'ProductSubcategory'] 
)
df_order_details = df_order_details[['Order ID', 'CategoryID', 'SubProductCategoryID', 'Amount', 'Quantity', 'Profit']]
df_order_details.to_sql('order_details', con=engine, if_exists='append', index=False, method='multi')


20000

In [36]:
df_sales_target

,Month of Order Date,Category,Target
36,Apr-2014,Electronics,15400.0
48,Apr-2015,Furniture,10896.0
60,Apr-2016,Furniture,5640.0
72,Apr-2017,Furniture,5932.0
0,Apr-2018,Furniture,10400.0
...,...,...,...
65,Sep-2016,Electronics,11303.0
77,Sep-2017,Clothing,15625.0
17,Sep-2018,Clothing,14000.0
29,Sep-2018,Electronics,9000.0


In [37]:
categories_df2 = categories_df[["ProductCategory", "CategoryID"]]
categories_df2 = categories_df2.drop_duplicates(subset=['ProductCategory'])
df_sales_target.dropna(how='all', inplace=True)
df_sales_target = df_sales_target.rename(columns={'Month of Order Date': 'MonthOfOrder'})
df_sales_target = df_sales_target.merge(
    categories_df2,
    how='left',
    left_on='Category',
    right_on='ProductCategory'
)
df_sales_target = df_sales_target[['CategoryID', 'MonthOfOrder', 'Target']]
df_sales_target['MonthOfOrder'] = pd.to_datetime(df_sales_target['MonthOfOrder'], infer_datetime_format=True).dt.strftime('%Y-%m-01')
df_sales_target.to_sql('sales_target', con=engine, if_exists='append', index=False, method='multi')

C:\Users\helto\AppData\Local\Temp\ipykernel_21520\840947685.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_sales_target['MonthOfOrder'] = pd.to_datetime(df_sales_target['MonthOfOrder'], infer_datetime_format=True).dt.strftime('%Y-%m-01')
C:\Users\helto\AppData\Local\Temp\ipykernel_21520\840947685.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_sales_target['MonthOfOrder'] = pd.to_datetime(df_sales_target['MonthOfOrder'], infer_datetime_format=True).dt.strftime('%Y-%m-01')


84

##### Execute queries

In [38]:
head_count = 5
query_times = []

for run in range(5):
    print(f"Run {run + 1}:\n")
    mysql_times = []
    
    with engine.connect() as connection:
        for q in queries:
            print(f"Executing: {q['description']}")
            start_time = time.time()
            
            if q['description'] == "Find all positive Profit in orders with less than 500 amount":
                query = """
                SELECT * FROM order_details od
                WHERE od.Profit > 0 AND od.Amount < 500 
                ORDER BY od.Profit DESC
                """
                result = connection.execute(text(query)).fetchall()
                
            elif q['description'] == "Find how many orders came from Mumbai":
                query = """
                SELECT COUNT(*) FROM orders_list ol
                WHERE ol.city = 'Mumbai'
                """
                result = connection.execute(text(query)).fetchall()
                
            elif q['description'] == "Identify Most Profitable States":
                query = """
                SELECT ol.State,
                    SUM(od.Profit) AS TotalProfit,
                    COUNT(od.`Order ID`) AS TotalOrders
                FROM order_details od
                JOIN orders_list ol ON od.`Order ID` = ol.`Order ID`
                GROUP BY ol.State
                HAVING TotalProfit > 0
                ORDER BY TotalProfit DESC;
                """
                result = connection.execute(text(query)).fetchall()
                
                df_state_profit = pd.DataFrame(result, columns=["State", "TotalProfit", "TotalOrders"])
                
                create_table_query = """
                CREATE TABLE IF NOT EXISTS state_profit (
                    State VARCHAR(255) PRIMARY KEY,
                    TotalProfit DECIMAL(12,2) NOT NULL,
                    TotalOrders INT NOT NULL
                )
                """
                connection.execute(text(create_table_query))
                df_state_profit.to_sql('state_profit', con=engine, if_exists='replace', index=False)

            elif q["description"] == "Monitor Sales Target Achievement Over Time":
                first_query = """
                WITH monthly_sales AS (
                    SELECT 
                        DATE_FORMAT(ol.`Order Date`, '%Y-%m') AS month,  
                        od.CategoryID,
                        SUM(od.Amount) AS total_sales
                    FROM order_details od
                    INNER JOIN orders_list ol ON od.`Order ID` = ol.`Order ID`
                    GROUP BY month, od.CategoryID
                )
                SELECT 
                    ms.month,
                    ms.CategoryID,
                    CASE 
                        WHEN ms.total_sales >= st.Target THEN 'Target achieved'
                        ELSE 'Target not achieved'
                    END AS target_met
                FROM monthly_sales ms
                INNER JOIN sales_target st
                    ON DATE_FORMAT(st.`MonthOfOrder`, '%Y-%m') = ms.month  
                    AND ms.CategoryID = st.CategoryID;
                """
                result = connection.execute(text(first_query)).fetchall()
                
                df_target = pd.DataFrame(result, columns=["month", "CategoryID", "target_met"])
                
                create_table_query = """
                CREATE TABLE IF NOT EXISTS performance_monitoring (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    month VARCHAR(10),
                    CategoryID INT,
                    target_met VARCHAR(20)
                )
                """
                connection.execute(text(create_table_query))
                df_target.to_sql('performance_monitoring', con=engine, if_exists='replace', index=False)
                
                second_query = """
                SELECT COUNT(*) 
                FROM performance_monitoring pm
                WHERE pm.target_met = 'Target achieved'
                """
                result = [f'{connection.execute(text(second_query)).fetchone()[0]} targets have been achieved']

            else:
                print(f"Unhandled query: {q['description']}")
                continue
            
            print(f"Results for '{q['description']}' (showing up to {head_count} results):")
            for i, doc in enumerate(result[:head_count]):
                print(doc)
            
            if len(result) > head_count:
                print(f"...and {len(result) - head_count} more results")
            
            end_time = time.time()
            mysql_times.append(end_time - start_time)
            print(f"Query executed in {end_time - start_time:.4f} seconds\n")
    
    query_times.append(mysql_times)

average_times = [sum(times) / len(times) for times in zip(*query_times)]

for i, q in enumerate(queries):
    print(f"Average execution time for '{q['description']}': {average_times[i]:.4f} seconds")
    

Run 1:

Executing: Find all positive Profit in orders with less than 500 amount
Results for 'Find all positive Profit in orders with less than 500 amount' (showing up to 5 results):
(2693, 'B-950416', 51, 3, 236, 74, 1000.0)
(8978, 'B-150914', 8, 1, 230, 38, 1000.0)
(3487, 'B-903647', 47, 1, 297, 68, 999.0)
(6495, 'B-682495', 30, 1, 408, 52, 999.0)
(8328, 'B-843715', 37, 2, 115, 70, 999.0)
...and 5444 more results
Query executed in 0.0274 seconds

Executing: Find how many orders came from Mumbai
Results for 'Find how many orders came from Mumbai' (showing up to 5 results):
(848,)
Query executed in 0.0070 seconds

Executing: Identify Most Profitable States
Results for 'Identify Most Profitable States' (showing up to 5 results):
('Bihar', 40908.0, 1083)
('Andhra Pradesh', 31567.0, 1049)
('Goa', 28017.0, 962)
('Rajasthan', 24938.0, 1028)
('Gujarat', 12094.0, 1087)
...and 4 more results
Query executed in 0.6979 seconds

Executing: Monitor Sales Target Achievement Over Time
Results for 'Mon

In [39]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM state_profit"))
    for row in result:
        print(row)

('Bihar', 40908.0, 1083)
('Andhra Pradesh', 31567.0, 1049)
('Goa', 28017.0, 962)
('Rajasthan', 24938.0, 1028)
('Gujarat', 12094.0, 1087)
('Haryana', 11240.0, 1029)
('Maharashtra', 3464.0, 1286)
('Delhi', 1724.0, 1030)
('West Bengal', 390.0, 1016)


In [40]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM performance_monitoring"))
    for row in result:
        print(row)

('2014-06', 1, 'Target achieved')
('2015-01', 1, 'Target achieved')
('2015-03', 1, 'Target achieved')
('2015-04', 1, 'Target achieved')
('2015-06', 1, 'Target achieved')
('2015-09', 1, 'Target achieved')
('2015-12', 1, 'Target achieved')
('2016-04', 1, 'Target achieved')
('2016-07', 1, 'Target achieved')
('2016-08', 1, 'Target achieved')
('2016-10', 1, 'Target achieved')
('2016-11', 1, 'Target achieved')
('2016-12', 1, 'Target achieved')
('2017-04', 1, 'Target achieved')
('2018-01', 1, 'Target achieved')
('2018-03', 1, 'Target achieved')
('2018-04', 1, 'Target not achieved')
('2018-05', 1, 'Target not achieved')
('2018-06', 1, 'Target not achieved')
('2018-07', 1, 'Target not achieved')
('2018-08', 1, 'Target achieved')
('2018-09', 1, 'Target not achieved')
('2018-10', 1, 'Target not achieved')
('2018-11', 1, 'Target achieved')
('2018-12', 1, 'Target achieved')
('2019-01', 1, 'Target achieved')
('2019-02', 1, 'Target achieved')
('2019-03', 1, 'Target achieved')
('2014-09', 2, 'Target a

### CREATING INDEXING ON MYSQL

In [41]:

with engine.connect() as connection:
    connection.execute(text("CREATE INDEX idx_profit_amount ON order_details(Profit, Amount)"))
    connection.execute(text("CREATE INDEX idx_city ON orders_list(City)"))
    connection.execute(text("CREATE INDEX idx_orders_list_state ON orders_list(State)"))
    connection.execute(text("CREATE INDEX idx_order_details_profit ON order_details(Profit)"))
    connection.execute(text("CREATE INDEX idx_order_date_month ON orders_list(`Order DATE`)"))
    connection.execute(text("CREATE INDEX idx_sales_target_category_month ON sales_target(CategoryID, `MonthOfOrder`)"))
    connection.execute(text("CREATE INDEX idx_order_details_category ON order_details(CategoryID)"))

In [42]:
head_count = 5
query_times = []

for run in range(5):
    print(f"Run {run + 1}:\n")
    mysql_times_ind = []
    
    with engine.connect() as connection:
        for q in queries:
            print(f"Executing: {q['description']}")
            start_time = time.time()
            
            if q['description'] == "Find all positive Profit in orders with less than 500 amount":
                query = """
                SELECT * FROM order_details od
                WHERE od.Profit > 0 AND od.Amount < 500 
                ORDER BY od.Profit DESC
                """
                result = connection.execute(text(query)).fetchall()
                
            elif q['description'] == "Find how many orders came from Mumbai":
                query = """
                SELECT COUNT(*) FROM orders_list ol
                WHERE ol.city = 'Mumbai'
                """
                result = connection.execute(text(query)).fetchall()
                
            elif q['description'] == "Identify Most Profitable States":
                query = """
                SELECT ol.State,
                    SUM(od.Profit) AS TotalProfit,
                    COUNT(od.`Order ID`) AS TotalOrders
                FROM order_details od
                JOIN orders_list ol ON od.`Order ID` = ol.`Order ID`
                GROUP BY ol.State
                HAVING TotalProfit > 0
                ORDER BY TotalProfit DESC;
                """
                result = connection.execute(text(query)).fetchall()
                
                df_state_profit = pd.DataFrame(result, columns=["State", "TotalProfit", "TotalOrders"])
                
                create_table_query = """
                CREATE TABLE IF NOT EXISTS state_profit (
                    State VARCHAR(255) PRIMARY KEY,
                    TotalProfit DECIMAL(12,2) NOT NULL,
                    TotalOrders INT NOT NULL
                )
                """
                connection.execute(text(create_table_query))
                df_state_profit.to_sql('state_profit', con=engine, if_exists='replace', index=False)

            elif q["description"] == "Monitor Sales Target Achievement Over Time":
                first_query = """
                WITH monthly_sales AS (
                    SELECT 
                        DATE_FORMAT(ol.`Order Date`, '%Y-%m') AS month,  
                        od.CategoryID,
                        SUM(od.Amount) AS total_sales
                    FROM order_details od
                    INNER JOIN orders_list ol ON od.`Order ID` = ol.`Order ID`
                    GROUP BY month, od.CategoryID
                )
                SELECT 
                    ms.month,
                    ms.CategoryID,
                    CASE 
                        WHEN ms.total_sales >= st.Target THEN 'Target achieved'
                        ELSE 'Target not achieved'
                    END AS target_met
                FROM monthly_sales ms
                INNER JOIN sales_target st
                    ON DATE_FORMAT(st.`MonthOfOrder`, '%Y-%m') = ms.month  
                    AND ms.CategoryID = st.CategoryID;
                """
                result = connection.execute(text(first_query)).fetchall()
                
                df_target = pd.DataFrame(result, columns=["month", "CategoryID", "target_met"])
                
                create_table_query = """
                CREATE TABLE IF NOT EXISTS performance_monitoring (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    month VARCHAR(10),
                    CategoryID INT,
                    target_met VARCHAR(20)
                )
                """
                connection.execute(text(create_table_query))
                df_target.to_sql('performance_monitoring', con=engine, if_exists='replace', index=False)
                
                second_query = """
                SELECT COUNT(*) 
                FROM performance_monitoring pm
                WHERE pm.target_met = 'Target achieved'
                """
                result = [f'{connection.execute(text(second_query)).fetchone()[0]} targets have been achieved']

            else:
                print(f"Unhandled query: {q['description']}")
                continue
            
            print(f"Results for '{q['description']}' (showing up to {head_count} results):")
            for i, doc in enumerate(result[:head_count]):
                print(doc)
            
            if len(result) > head_count:
                print(f"...and {len(result) - head_count} more results")
            
            end_time = time.time()
            mysql_times_ind.append(end_time - start_time)
            print(f"Query executed in {end_time - start_time:.4f} seconds\n")
    
    query_times.append(mysql_times_ind)

average_times = [sum(times) / len(times) for times in zip(*query_times)]

for i, q in enumerate(queries):
    print(f"Average execution time for '{q['description']}': {average_times[i]:.4f} seconds")

Run 1:

Executing: Find all positive Profit in orders with less than 500 amount
Results for 'Find all positive Profit in orders with less than 500 amount' (showing up to 5 results):
(2693, 'B-950416', 51, 3, 236, 74, 1000.0)
(8978, 'B-150914', 8, 1, 230, 38, 1000.0)
(3487, 'B-903647', 47, 1, 297, 68, 999.0)
(6495, 'B-682495', 30, 1, 408, 52, 999.0)
(8328, 'B-843715', 37, 2, 115, 70, 999.0)
...and 5444 more results
Query executed in 0.0401 seconds

Executing: Find how many orders came from Mumbai
Results for 'Find how many orders came from Mumbai' (showing up to 5 results):
(848,)
Query executed in 0.0030 seconds

Executing: Identify Most Profitable States
Results for 'Identify Most Profitable States' (showing up to 5 results):
('Bihar', 40908.0, 1083)
('Andhra Pradesh', 31567.0, 1049)
('Goa', 28017.0, 962)
('Rajasthan', 24938.0, 1028)
('Gujarat', 12094.0, 1087)
...and 4 more results
Query executed in 0.2805 seconds

Executing: Monitor Sales Target Achievement Over Time
Results for 'Mon

## Comparison of execution times

In [43]:
comparison_df = pd.DataFrame({
    "Query Description": [q['description'] for q in queries],
    "MongoDB_Time(s)": mongo_times,
    "MongoDB_Time(s) after Indexing": mongo_times_ind,
    "MySQL_Time(s)": mysql_times,
    "MySQL_Time(s) after Indexing": mysql_times_ind,
    
})

# comparison_df["Difference (MongoDB - MySQL) (s)"] = comparison_df["MongoDB_Time(s)"] - comparison_df["MySQL_Time(s)"]
comparison_df["Faster Database"] = comparison_df.apply(
    lambda row: "MongoDB" if row["MongoDB_Time(s)"] < row["MySQL_Time(s)"] else "MySQL",
    axis=1
)

# comparison_df["Difference (MongoDB - MySQL) (s), with index"] = comparison_df["MongoDB_Time(s) after Indexing"] - comparison_df["MySQL_Time(s) after Indexing"]
comparison_df["Faster Database with index"] = comparison_df.apply(
    lambda row: "MongoDB" if row["MongoDB_Time(s) after Indexing"] < row["MySQL_Time(s) after Indexing"] else "MySQL",
    axis=1
)

comparison_df


,Query Description,MongoDB_Time(s),MongoDB_Time(s) after Indexing,MySQL_Time(s),MySQL_Time(s) after Indexing,Faster Database,Faster Database with index
0,Find all positive Profit in orders with less t...,0.094588,0.096425,0.033154,0.040983,MySQL,MySQL
1,Find how many orders came from Mumbai,0.020381,0.021076,0.006065,0.006001,MySQL,MySQL
2,Identify Most Profitable States,0.135018,0.104841,0.142398,0.250285,MongoDB,MongoDB
3,Monitor Sales Target Achievement Over Time,0.231617,0.174782,0.347379,0.292775,MongoDB,MongoDB
